<a href="https://colab.research.google.com/github/hritik880/Custom-OCR-/blob/master/Aadhar_opencv_cropping_and_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pillow
!pip install pytesseract
!sudo apt install tesseract-ocr

  Created wheel for pytesseract: filename=pytesseract-0.3.4-py2.py3-none-any.whl size=13431 sha256=381f03a58a98381ad4e5012e7b103e204c03a152d2787d29ae4abab166f10df3
  Stored in directory: /root/.cache/pip/wheels/63/2a/a0/7596d2e0a73cf0aeffd6f6170862c4e73f3763b7827e48691a
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 33 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 

In [3]:
!unzip /content/drive/'My Drive'/data_for_colab.zip

Archive:  /content/drive/My Drive/data_for_colab.zip
   creating: data_for_colab/
  inflating: data_for_colab/.DS_Store  
   creating: data_for_colab/data/
  inflating: data_for_colab/data/a1.png  
  inflating: data_for_colab/data/a1.txt  
  inflating: data_for_colab/data/a10.png  
  inflating: data_for_colab/data/a10.txt  
  inflating: data_for_colab/data/a11.png  
  inflating: data_for_colab/data/a11.txt  
  inflating: data_for_colab/data/a12.png  
  inflating: data_for_colab/data/a12.txt  
  inflating: data_for_colab/data/a13.png  
  inflating: data_for_colab/data/a13.txt  
  inflating: data_for_colab/data/a14.png  
  inflating: data_for_colab/data/a14.txt  
  inflating: data_for_colab/data/a15.png  
  inflating: data_for_colab/data/a15.txt  
  inflating: data_for_colab/data/a16.png  
  inflating: data_for_colab/data/a16.txt  
  inflating: data_for_colab/data/a17.png  
  inflating: data_for_colab/data/a17.txt  
  inflating: data_for_colab/data/a18.png  
  inflating: data_for_colab/d

In [4]:
!cp /content/drive/'My Drive'/classes.txt /content/data_for_colab

In [5]:
import cv2
import numpy as np
import pytesseract
import pandas as pd
from google.colab.patches import cv2_imshow
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

In [6]:
def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


In [21]:
from PIL import Image
weights_path = '/content/drive/My Drive/yolov3_7000.weights'
config_path = '/content/data_for_colab/yolov3.cfg'
classes_path = '/content/data_for_colab/classes.txt'
data = []

for j in range(1,4):
  aadhar_data = {}
  image_path = '/content/data_for_colab/data/a%s.png'%j
  image = cv2.imread(image_path)
  # cv2_imshow(image) 
  Width = image.shape[1]
  Height = image.shape[0]
  scale = 0.00392

  classes = None

  with open(classes_path, 'r') as f:
      classes = [line.strip() for line in f.readlines()]

  COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

  net = cv2.dnn.readNet(weights_path, config_path)

  blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

  net.setInput(blob)

  outs = net.forward(get_output_layers(net))

  class_ids = []
  confidences = []
  boxes = []
  conf_threshold = 0.5
  nms_threshold = 0.4


  for out in outs:
      for detection in out:
          scores = detection[5:]
          class_id = np.argmax(scores)
          confidence = scores[class_id]
          if confidence > 0.5:
              center_x = int(detection[0] * Width)
              center_y = int(detection[1] * Height)
              w = int(detection[2] * Width)
              h = int(detection[3] * Height)
              x = center_x - w / 2
              y = center_y - h / 2
              class_ids.append(class_id)
              confidences.append(float(confidence))
              boxes.append([x, y, w, h])

  im_pil = Image.fromarray(image)

  indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

  for i in indices:
      i = i[0]
      box = boxes[i]
      x = box[0]
      y = box[1]
      w = box[2]
      h = box[3]
      #draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
      cropped_img = im_pil.crop((x,y,x+w,y+h))
      im_np = np.asarray(cropped_img)
      sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
      img = cv2.filter2D(im_np, -1, sharpen_kernel)
      scale_percent =300 # percent of original size
      width = int(img.shape[1] * scale_percent / 100) 
      height = int(img.shape[0] * scale_percent / 100)
      dim = (width, height)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      text = pytesseract.image_to_string(img)
      if(class_ids[i]==0):
        aadhar_data['Name'] = text
      elif(class_ids[i]==1):
        aadhar_data['DOB'] = text
      elif(class_ids[i]==2):
        aadhar_data['Gender'] = text
      else:
        aadhar_data['Aadhar'] = text
      # cv2_imshow(img) #uncomment cv2_imshow if you want to see cropped image
      # cv2.waitKey()
  data.append(aadhar_data)
dataFrame = pd.DataFrame.from_dict(data)
dataFrame.to_csv('aadhar_data_sharpen.csv', index=False)
